## Import Libraries

In [29]:
import requests, re, time, random
import datetime as dt
import pandas as pd 
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm_notebook

In [34]:
driver = webdriver.Chrome('C:/chrome/chromedriver.exe')

## Setting Constant

In [38]:
KEYWORD = '집+인테리어'
START_DATE = dt.date(year=2019, month=5, day=1)
UNTIL_DATE = dt.date(year=2019, month=5, day=2)
END_DATE = dt.date(year=2019, month=6, day=2)
SLEEP_RANDOM = random.randint(1,3)
GET_HEIGHT = "return document.body.scrollHeight"
GET_LAST_HEIGHT = "return $(document).height() - $(window).height();"
MOVE_SCROLL = "window.scrollTo(0, document.body.scrollHeight);"

In [39]:
def parse() :
    html = driver.page_source
    soup = bs(html, 'html.parser')  
    
    return soup

def control(driver, statement) :

    if (statement == GET_HEIGHT) | (statement == GET_LAST_HEIGHT) :
        result = driver.execute_script(statement)
        return result

    else : 
        driver.execute_script(statement)

## Crawling Code

In [40]:
from tqdm import tqdm_notebook

tweet_text = [] 

while END_DATE != START_DATE :
    URL = f'https://twitter.com/search?l=&q={KEYWORD}%20since%3A{str(START_DATE)}%20until%3A{str(UNTIL_DATE)}&src=typd'
    driver.get(URL)
    time.sleep(SLEEP_RANDOM)
    
    daily_tw = {'Date':START_DATE}
    tweets = []
    currHeight = control(driver, GET_HEIGHT)
    
    while True : 
        soup = parse()
        
        tmp = soup.find_all("p", {"class" : "TweetTextSize"})
        tweets.extend(tmp)
        
        control(driver, MOVE_SCROLL)
        time.sleep(SLEEP_RANDOM)    

        lastHeight = control(driver, GET_HEIGHT) #현재 스크롤 기준 마지막 높이


        if lastHeight == currHeight : #현재 스크롤이 움직임이 더이상 없다면 맨 끝에 위치했다는 것. 
            daily_tw['tweets'] = tweets
            tweet_text.append(daily_tw)
            START_DATE = UNTIL_DATE
            UNTIL_DATE += dt.timedelta(days=1) 
            
            break
            
        currHeight = lastHeight

한글만 남기기

In [41]:
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

hangul_tweet = {}

for t in tweet_text :
    hangul_tweet = hangul.sub('', str(t['tweets'])) # 한글과 띄어쓰기를 제외한 모든 부분을 제거

print(hangul_tweet[:100])

     친구 둘이 살기 딱 좋은 구조                      구해줘홈즈       제주도       인테리어      엄마네 집 근처에 있는 카페갬성적인 인테리어가


In [42]:
data_file = 'tw_todayhouse.data'

with open(data_file, 'w', encoding='utf=8') as fp :
    fp.write("".join(hangul_tweet))